In [1]:
#lstm2

In [2]:
#lstm module
from sklearn.preprocessing import MinMaxScaler
import pandas as pd 
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Activation, Input
from mlflow.models import infer_signature
import mlflow
import mlflow.keras
from datetime import datetime
import argparse
from keras.optimizers import Adam
import os 
from sklearn.metrics import accuracy_score, classification_report
# from sklearn.metrics import accuracy_score, classification_report


class Lstm:
    def __init__(self, data, ticker, frequency, target, timestamps, model_name, SEQ_LENGTH=100):
        self.data = data
        self.ticker = ticker
        self.frequency = frequency
        self.target = target
        self.timestamps = timestamps
        self.model_name = model_name
        self.SEQ_LENGTH = SEQ_LENGTH
    
    def getData(self):

        df = self.data
        scaler = MinMaxScaler()
        self.scaler = scaler
        close_price = df[self.target].values.reshape(-1, 1)

        scaled_close = scaler.fit_transform(close_price)
        # SEQ_LEN = 100
        self.X_train, self.y_train, self.X_test, self.y_test = preprocess(scaled_close, self.SEQ_LENGTH, train_split = 0.95)


    # def compileModel(self, opt='adam', loss='mean_squared_error'):
    #     #We’re creating a 3 layer LSTM Recurrent Neural Network. We use Dropout with a rate of 20% to combat overfitting during training:

    #     # Set your constants
    #     DROPOUT = 0.2 
    #     WINDOW_SIZE = self.SEQ_LENGTH - 1

    #     # Define the model
    #     self.model = Sequential()

    #     self.model.add(Bidirectional(
    #         LSTM(WINDOW_SIZE, return_sequences=True),  # Use LSTM directly
    #         input_shape=(WINDOW_SIZE, self.X_train.shape[-1])
    #     ))
    #     self.model.add(Dropout(rate=DROPOUT))

    #     # Add more layers as needed...
    #     self.model.add(Dense(1))  # Example output layer

    #     # Compile the model
    #     self.model.compile(optimizer=opt, loss=loss)

    #     # Summary of the model
    #     self.model.summary()

    #     self.model.add(Activation('linear'))

    def compileModel(self, opt='adam', loss='mean_squared_error', learning_rate=0.0001):
        # Set your constants
        DROPOUT = 0.2 
        WINDOW_SIZE = self.SEQ_LENGTH - 1

        # Define the model
        self.model = Sequential()

        # Add Input layer
        self.model.add(Input(shape=(WINDOW_SIZE, self.X_train.shape[-1])))

        # Add LSTM layer with Bidirectional wrapper
        self.model.add(Bidirectional(LSTM(WINDOW_SIZE, return_sequences=True)))

        # Add Dropout layer
        self.model.add(Dropout(rate=DROPOUT))

        # Add more layers as needed...
        self.model.add(Dense(1))  # Example output layer

        # Define the optimizer with the specified learning rate
        if opt == 'adam':
            optimizer = Adam(learning_rate=learning_rate)
        else:
            optimizer = opt  # Use other optimizers if specified

        # Compile the model
        self.model.compile(optimizer=optimizer, loss=loss)

        # Add Activation layer (if necessary, usually added in the Dense layer for the output)
        self.model.add(Activation('linear'))  # This line might not be needed since Activation can be in Dense

        # Summary of the model
        print(self.model.summary())

    # def trainModel(self):

    #     self.today = datetime.now().strftime("%Y-%m-%d")

    #     # Set the experiment name (optional)
    #     mlflow.set_experiment(f"{self.ticker}_{self.frequency}_{self.target}_experiment")

    #     BATCH_SIZE = 32

    #     with mlflow.start_run():
    #         # Compile the model
    #         # self.model.compile(
    #         #     loss='mean_squared_error',
    #         #     optimizer='adam'
    #         # )

    #         # Log parameters
    #         mlflow.log_param("batch_size", BATCH_SIZE)
    #         mlflow.log_param("epochs", 15)
    #         mlflow.log_param("optimizer", "adam")

    #         history = self.model.fit(
    #             self.X_train,
    #             self.y_train,
    #             epochs=15,
    #             batch_size=BATCH_SIZE,
    #             shuffle=False,
    #             validation_split=0.1
    #         )

    #         # Log metrics
    #         mlflow.log_metric("final_loss", history.history['loss'][-1])
    #         mlflow.log_metric("final_val_loss", history.history['val_loss'][-1])

    # def saveModel(self):

    #     # After your model training and before logging the model
    #     signature = infer_signature(self.X_train, self.model.predict(self.X_train))

    #     with mlflow.start_run():
    #         mlflow.keras.log_model(self.model, "model", signature=signature)
    #         # Save the model locally
    #         self.model.save(f"models/lstm/{self.ticker}_{self.frequency}_{self.target}_{self.today}.h5")
    def trainModel(self):
        self.model_date = datetime.now().date().strftime("%Y-%m-%d")

        # Set the experiment name (optional)
        mlflow.set_experiment(f"{self.ticker}_{self.frequency}_{self.target}_{self.model_date}_lstm_experiment")

        BATCH_SIZE = 32

        with mlflow.start_run():
            # Log parameters
            mlflow.log_param("batch_size", BATCH_SIZE)
            mlflow.log_param("epochs", 15)
            mlflow.log_param("optimizer", "adam")

            # Train the model
            history = self.model.fit(
                self.X_train,
                self.y_train,
                epochs=15,
                batch_size=BATCH_SIZE,
                shuffle=False,
                validation_split=0.1
            )

            # Log metrics
            mlflow.log_metric("final_loss", history.history['loss'][-1])
            mlflow.log_metric("final_val_loss", history.history['val_loss'][-1])

    def saveModel(self):
        # Predict on validation/test data
        y_pred = self.model.predict(self.X_test)
        # Convert predictions to class labels if necessary
        y_pred_classes = (y_pred > 0.5).astype(int)  # Example for binary classification

        # Calculate accuracy (or any other relevant metrics)
        accuracy = accuracy_score(self.y_test, y_pred_classes)

        # Create a model directory
        model_name = f"{self.ticker}_{self.frequency}_{self.target}_{self.model_date}"
        model_dir = f"models/lstm/{model_name}"
        os.makedirs(model_dir, exist_ok=True)

        # Save the model as an .h5 file
        model_path = os.path.join(model_dir, f"{model_name}.h5")
        self.model.save(model_path)

        # Log the model and metrics with MLflow
        with mlflow.start_run():
            # mlflow.keras.log_model(self.model, "model")
            mlflow.log_metric("accuracy", accuracy)

            # Save metrics to a text file
            metrics_path = os.path.join(model_dir, "metrics.txt")
            with open(metrics_path, "w") as f:
                f.write(f"Final Accuracy: {accuracy}\n")
                f.write("Classification Report:\n")
                f.write(classification_report(self.y_test, y_pred_classes))

        print(f'LSTM Model saved to {model_path} with accuracy: {accuracy}')

    def predictAndUnscale(self):
        self.y_pred = self.model.predict(self.X_test)
        # Select the last timestep
        # Assuming y_hat has shape (batch_size, timesteps, features)
        # Select the last timestep for predictions
        self.y_pred_last_timestamp = self.y_pred[:, -1, :]  # Make sure y_hat has the expected shape

        # Inverse transform the predictions
        self.y_pred_inverse = self.scaler.inverse_transform(self.y_pred_last_timestamp)

        # Assuming y_test is a 2D array (if you're using a sequence of values)
        # If y_test was reshaped correctly when prepared, do this:
        self.y_test_inverse = self.scaler.inverse_transform(self.y_test)

        if self.y_test.ndim == 3:
            self.y_test_last_timestep = self.y_test[:, -1, :]  # Only if y_test is 3D
            self.y_test_inverse = self.scaler.inverse_transform(self.y_test_last_timestep)

    def setModel(self, model):
        self.model = model

        

def to_sequences(data, seq_len):
    d = []

    for index in range(len(data) - seq_len):
        d.append(data[index: index + seq_len])

    return np.array(d)

def preprocess(data_raw, seq_len, train_split):

    data = to_sequences(data_raw, seq_len)

    num_train = int(train_split * data.shape[0])

    X_train = data[:num_train, :-1, :]
    y_train = data[:num_train, -1, :]

    X_test = data[num_train:, :-1, :]
    y_test = data[num_train:, -1, :]

    return X_train, y_train, X_test, y_test




In [3]:
ticker = "ETHUSD"
frq = "1"
target = "return_8n"
# parser = argparse.ArgumentParser(description='Process cryptocurrency data.')
# parser.add_argument('ticker', type=str, help='The cryptocurrency ticker symbol (e.g., BTC, ETH)')
# parser.add_argument('frequency', type=int, help='The frequency of data points (e.g., 1 for daily, 7 for weekly)')
# parser.add_argument('target', type=str, help='target return period (e.g return_n8 (return for period n + 8), 2^i')


# args = parser.parse_args()
# ticker = args.ticker
# frq = args.frequency
# target = args.target

df = pd.read_csv(f"data/silver_prices/{ticker}_{frq}_silver.csv")

df = df[-int((len(df)/2)):]

print(df)
timestamps = list(df['timestamp'])
model_date = datetime.now().date().strftime("%Y-%m-%d")
#
model_name = f"{ticker}_{frq}_{target}_{model_date}"
model_path = "models/lstm/"+model_name

lstm = Lstm(df, ticker, frq, target, timestamps, model_name, SEQ_LENGTH=100)

lstm.getData()
lstm.compileModel()
lstm.trainModel()
# lstm.saveModel()
# lstm.predictAndUnscale()
# preds = lstm.y_pred_inverse
# real = lstm.y_test_inverse
# data = {
#     "timestamps": timestamps,
#     "predicted":preds,
#     "real":real,
# }
# result = pd.DataFrame(data)
# result.to_csv(f"models/model-output/lstm/{model_name}.csv")

           timestamp     open     high      low    close     volume  \
136824  1.718736e+09  3414.42  3414.42  3414.32  3414.32   0.221110   
136825  1.718736e+09  3413.68  3413.68  3413.68  3413.68   1.432814   
136826  1.718736e+09  3413.68  3413.68  3410.28  3410.28  13.196809   
136827  1.718736e+09  3410.60  3410.60  3407.70  3407.70   0.579290   
136828  1.718736e+09  3407.71  3407.71  3407.71  3407.71   5.829947   
...              ...      ...      ...      ...      ...        ...   
273642  1.727740e+09  2597.52  2598.13  2597.52  2598.13   0.714424   
273643  1.727741e+09  2598.64  2600.99  2598.64  2600.99   0.985735   
273644  1.727741e+09  2601.93  2601.93  2601.93  2601.93   0.105847   
273645  1.727741e+09  2602.25  2603.48  2602.24  2603.48   5.066109   
273646  1.727741e+09  2602.97  2602.97  2602.97  2602.97   0.323397   

                   0            1          2            3  ...  \
136824  1.718736e+09  3416.984350  3411.1750  3405.365650  ...   
136825  1.71873

2024-11-02 17:48:29.871723: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-11-02 17:48:29.871752: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-11-02 17:48:29.871757: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-11-02 17:48:29.871940: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-02 17:48:29.871951: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 99, 198)        │        79,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 99, 198)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 99, 1)          │           199 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 99, 1)          │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,191 (313.25 KB)

...

 Non-trainable params: 0 (0.00 B)

2024/11/02 17:48:30 INFO mlflow.tracking.fluent: Experiment with name 'ETHUSD_1_return_8n_2024-11-02_lstm_experiment' does not exist. Creating a new experiment.


None
Epoch 1/15


2024-11-02 17:48:30.548134: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


3654/3654 ━━━━━━━━━━━━━━━━━━━━ 217s 58ms/step - loss: 0.0164 - val_loss: 6.3517e-04
Epoch 2/15
3654/3654 ━━━━━━━━━━━━━━━━━━━━ 214s 59ms/step - loss: 8.7251e-04 - val_loss: 1.1213e-04
Epoch 3/15
3654/3654 ━━━━━━━━━━━━━━━━━━━━ 218s 60ms/step - loss: 4.2710e-04 - val_loss: 7.4598e-05
Epoch 4/15
3654/3654 ━━━━━━━━━━━━━━━━━━━━ 218s 60ms/step - loss: 3.4911e-04 - val_loss: 7.4345e-05
Epoch 5/15
3654/3654 ━━━━━━━━━━━━━━━━━━━━ 221s 61ms/step - loss: 3.0634e-04 - val_loss: 7.3730e-05
Epoch 6/15
3654/3654 ━━━━━━━━━━━━━━━━━━━━ 223s 61ms/step - loss: 2.7303e-04 - val_loss: 7.9748e-05
Epoch 7/15
2419/3654 ━━━━━━━━━━━━━━━━━━━━ 1:12 59ms/step - loss: 2.3083e-04

In [ ]:
lstm.saveModel()


# lstm.predictAndUnscale()
# preds = lstm.y_pred_inverse
# real = lstm.y_test_inverse
# data = {
#     "timestamps": timestamps,
#     "predicted":preds,
#     "real":real,
# }
# result = pd.DataFrame(data)
# result.to_csv(f"models/model-output/lstm/{model_name}.csv")

228/228 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step


ValueError: Input y_true contains NaN.